# Realización de operaciones de base de datos en marcos de datos

## Acerca de los datos
En este NoteBook, utilizaremos datos meteorológicos diarios extraídos de la base de datos[National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). En [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) contiene el proceso que se ha seguido para recoger los datos. Consulte la ficha del conjunto de datos [documentation](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf) para obtener información sobre los campos.

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*


## Setup

In [1]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


## Consulta de DataFrames
El método `query()` es una forma más sencilla de filtrar basándose en algunos criterios. Por ejemplo, podemos usarlo para encontrar todas las entradas en las que se registró nieve de una estación con `US1NY` en su ID de estación:

In [2]:
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")')
snow_data.head()

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
789,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0007,",,N,",41.0
794,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0018,",,N,",10.0
798,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0024,",,N,",89.0
800,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0030,",,N,",102.0


Esto equivale a consultar la base de datos SQLite `weather.db` en busca de

```sql
SELECT * 
FROM weather 
WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"
```

In [3]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 and station LIKE "%US1NY%"', 
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

True

Tenga en cuenta que esto también equivale a crear máscaras booleanas:

In [4]:
weather[
    (weather.datatype == 'SNOW') 
    & (weather.value > 0)
    & weather.station.str.contains('US1NY')
].equals(snow_data)

True

## Merging DataFrames
Tenemos datos de muchas estaciones diferentes cada día; sin embargo, no sabemos cuáles son las estaciones&mdash;sólo sus IDs. Podemos unir los datos del archivo `weather_stations.csv` que contiene información del endpoint `stations` de la API del NCEI. Consulte el archivo[`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) para ver cómo se recogió esto. Tiene este aspecto:

In [5]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6


A modo de recordatorio, los datos meteorológicos tienen este aspecto:

In [6]:
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


Podemos unir nuestros datos haciendo coincidir la columna `station_info.id` con la columna `weather.station`. Antes de hacer eso, sin embargo, vamos a ver cuántos valores únicos que tenemos:

In [7]:
station_info.id.describe()

count                   279
unique                  279
top       GHCND:US1NJBG0029
freq                      1
Name: id, dtype: object

Mientras que `station_info` tiene una fila por estación, el marco de datos `weather` tiene muchas entradas por estación. Observa que también tiene menos uniques:

In [8]:
weather.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

Al trabajar con uniones, es importante vigilar el recuento de filas. Algunos tipos de join provocan pérdidas de datos. Recuerda que podemos conseguirlo con `shape`:

In [9]:
station_info.shape[0], weather.shape[0]

(279, 78780)

Dado que vamos a hacer esto a menudo, tiene más sentido escribir una función:

In [10]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

[279, 78780]

Por defecto, `merge()` realiza una unión interna. Basta con especificar las columnas que se utilizarán para la unión. El marco de datos izquierdo es sobre el que llamamos a `merge()`, y el derecho se pasa como argumento:

In [11]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=0)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
10739,2018-08-07T00:00:00,SNOW,GHCND:US1NJMN0069,",,N,",0.0,GHCND:US1NJMN0069,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
45188,2018-12-21T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",16.7,GHCND:USW00014732,"LAGUARDIA AIRPORT, NY US",40.779440,-73.880350,3.4
59823,2018-01-15T00:00:00,WDF5,GHCND:USW00094741,",,W,",40.0,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",40.850000,-74.061390,2.7
10852,2018-10-31T00:00:00,PRCP,GHCND:US1NJMN0069,"T,,N,",0.0,GHCND:US1NJMN0069,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
46755,2018-05-05T00:00:00,SNOW,GHCND:USW00014734,",,W,",0.0,GHCND:USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.682500,-74.169400,2.1


Podemos eliminar la duplicación de información en las columnas `station` e `id` renombrando una de ellas antes de la fusión y utilizando simplemente `on`:

In [12]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)

,date,datatype,station,attributes,value,name,latitude,longitude,elevation
10739,2018-08-07T00:00:00,SNOW,GHCND:US1NJMN0069,",,N,",0.0,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
45188,2018-12-21T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",16.7,"LAGUARDIA AIRPORT, NY US",40.779440,-73.880350,3.4
59823,2018-01-15T00:00:00,WDF5,GHCND:USW00094741,",,W,",40.0,"TETERBORO AIRPORT, NJ US",40.850000,-74.061390,2.7
10852,2018-10-31T00:00:00,PRCP,GHCND:US1NJMN0069,"T,,N,",0.0,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
46755,2018-05-05T00:00:00,SNOW,GHCND:USW00014734,",,W,",0.0,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.682500,-74.169400,2.1


Estamos perdiendo estaciones que no tienen observaciones meteorológicas asociadas, si no queremos perder estas filas, realizamos un join derecho o izquierdo en lugar del join interno:

In [13]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join[right_join.datatype.isna()].head()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
0,NaN,NaN,NaN,NaN,NaN,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
344,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
345,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
718,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0005,"WESTWOOD 0.8 ESE, NJ US",40.983041,-74.015858,15.8
719,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0006,"RAMSEY 0.6 E, NJ US",41.058611,-74.134068,112.2


Las uniones a la izquierda y a la derecha que hemos realizado anteriormente son equivalentes porque el lado para el que guardamos las filas sin coincidencias era el mismo en ambos casos:

In [14]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True)
)

True

Observe que tenemos filas adicionales en las uniones izquierda y derecha porque conservamos todas las estaciones que no tenían observaciones meteorológicas:

In [15]:
get_row_count(inner_join, left_join, right_join)

[78780, 78949, 78949]

Si consultamos la información de las estaciones que tienen `"US1NY"` en su `id` y realizamos una unión externa, podemos ver dónde se producen los desajustes:

In [16]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation,_merge
23634,2018-04-12T00:00:00,PRCP,GHCND:US1NYNS0043,",,N,",0.0,GHCND:US1NYNS0043,"PLAINVIEW 0.4 ENE, NY US",40.785919,-73.466873,56.7,both
25742,2018-03-25T00:00:00,PRCP,GHCND:US1NYSF0061,",,N,",0.0,GHCND:US1NYSF0061,"CENTERPORT 0.9 SW, NY US",40.891689,-73.383133,53.6,both
60645,2018-04-16T00:00:00,TMIN,GHCND:USW00094741,",,W,",3.9,NaN,NaN,NaN,NaN,NaN,left_only
70764,2018-03-23T00:00:00,SNWD,GHCND:US1NJHD0002,",,N,",203.0,NaN,NaN,NaN,NaN,NaN,left_only
78790,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYQN0033,"HOWARD BEACH 0.4 NNW, NY US",40.662099,-73.841345,2.1,right_only
78800,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYWC0009,"NEW ROCHELLE 1.3 S, NY US",40.904000,-73.777000,21.9,right_only


Estas uniones son equivalentes a sus equivalentes SQL. A continuación se muestra la unión interna. Tenga en cuenta que para utilizar `equals()` tendrá que hacer alguna manipulación de los marcos de datos para alinearlos:

In [17]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    inner_join_from_db = pd.read_sql(
        'SELECT * FROM weather JOIN stations ON weather.station == stations.id', 
        connection
    )

inner_join_from_db.shape == inner_join.shape

True

Revisando los datos sucios del capítulo 3 [`5-handling_data_issues.ipynb`](../ch_03/5-handling_data_issues.ipynb)

Significado de los datos:
- `PRCP`: precipitación en milímetros
- `SNOW`: nevada en milímetros
- SNWD`: profundidad de la nieve en milímetros
- TMAX`: temperatura máxima diaria en grados Celsius
- TMIN`: temperatura mínima diaria en grados Celsius
- TOBS`: temperatura en el momento de la observación en grados Celsius
- WESF: equivalente en agua de la nieve en milímetros


Lee los datos, eliminando los duplicados y la columna poco informativa `SNWD`:

In [18]:
dirty_data = pd.read_csv(
    'data/dirty_data.csv', index_col='date'
).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04T00:00:00,?,20.6,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05T00:00:00,?,0.3,NaN,5505.0,-40.0,NaN,NaN,NaN


Necesitamos crear dos marcos de datos para la unión. También eliminaremos algunas columnas innecesarias para facilitar la visualización:

In [19]:
valid_station = dirty_data.query('station != "?"').drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

Nuestra columna para la unión es el índice en ambos marcos de datos, por lo que debemos especificar `left_index` y `right_index`:

In [20]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True
).query('WESF > 0').head()

,PRCP_x,SNOW_x,TMAX,TMIN,TOBS,inclement_weather_x,PRCP_y,SNOW_y,WESF,inclement_weather_y
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


A las columnas que existían en ambos marcos de datos, pero que no formaban parte de la unión, se les añadieron sufijos a sus nombres: `_x` para las columnas del marco de datos izquierdo e `_y` para las columnas del marco de datos derecho. Podemos personalizar esto con el argumento `suffixes`:

In [21]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True, suffixes=('', '_?')
).query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


Dado que estamos uniendo en el índice, una forma más fácil es utilizar el método `join()` en lugar de `merge()`. Observe que el parámetro sufijo es ahora `lsuffix` para el sufijo del marco de datos izquierdo y `rsuffix` para el derecho:

In [22]:
valid_station.join(station_with_wesf, how='left', rsuffix='_?').query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


Las uniones pueden consumir muchos recursos, por lo que es una buena idea averiguar qué tipo de unión necesitamos utilizando operaciones de conjunto antes de intentar la unión en sí. Las operaciones de conjunto `pandas` se realizan sobre el índice, por lo que las columnas sobre las que vayamos a realizar la unión tendrán que ser el índice. Volvamos a los marcos de datos `weather` y `station_info` y establezcamos las columnas de ID de estación como índice:

In [23]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

La intersección nos dirá las estaciones que están presentes en ambos marcos de datos. El resultado será el índice al realizar una unión interna:

In [24]:
weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

La diferencia de conjuntos nos dirá lo que perdemos de cada lado. Al realizar una unión interna, no perdemos nada del marco de datos `weather`:

In [25]:
weather.index.difference(station_info.index)

Index([], dtype='object')

Sin embargo, perdemos 169 estaciones del marco de datos `station_info`:

In [26]:
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

La diferencia simétrica nos dice lo que perdemos por ambos lados. Es la combinación de las diferencias de conjunto en cada dirección:

In [27]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name.index).shape[0]

True

La unión nos mostrará todo lo que habrá después de una unión externa completa. Tenga en cuenta que pasamos los valores únicos del índice para asegurarnos de que podemos ver el número de estaciones que nos quedarán:

In [28]:
weather.index.unique().union(station_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=279)

Obsérvese que la diferencia simétrica es en realidad la unión de las diferencias de conjunto:

In [29]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).union(weather.index.difference(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

True

<hr>

<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="text-align: left;">
        <a href="../ch_03/5-handling_data_issues.ipynb">
            <button>&#8592; Capitulo 3</button>
        </a>
    </div>
    <div style="text-align: center;">
        <a href="./0-weather_data_collection.ipynb">
            <button>Weather Data Collection</button>
        </a>
    </div>
    <div style="text-align: right;">
        <a href="./2-operaciones_con_dataframe.ipynb">
            <button>Proximo Notebook &#8594;</button>
        </a>
    </div>
</div>

<hr>
